In [5]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
import numpy as np
import pandas as pd
import json

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding
from tensorflow.python.keras.layers import CuDNNGRU
from tensorflow.keras.optimizers import Adam

In [7]:
dataframe = pd.read_csv('/content/gdrive/MyDrive/MachineLearningProject/newDataset.csv', sep = ",")
target = dataframe['Puan']
data = dataframe['reviews.text'].astype(str)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
target_int = [int(index) for index in target]

In [9]:
cutoff = int(len(data) * 0.85)
x_train, x_test = data[:cutoff], data[cutoff:]
y_train, y_test = target_int[:cutoff], target_int[cutoff:]

x_test

29461    This device is very easy to set up as long as ...
29462    Best product ever!!!!! I would buy another and...
29463    works very well, small but not too small, very...
29464    Box works faster than the stick as long as you...
29465    Echo works well with a variety of different de...
                               ...                        
34655    Pros: 5 Ghz wifi, supports expansion cardsCons...
34656    Both my 2 year old and my 4 year old love this...
34657    Love the child profiles for my kids and all th...
34658    It's worth the $50. The amazon App Store has a...
34659    The fire meets my needs for my reading enjoyme...
Name: reviews.text, Length: 5199, dtype: object

In [10]:
with open('/content/gdrive/MyDrive/MachineLearningProject/tokenizer.json') as json_dosyasi:
    json_tokenizer = json.load(json_dosyasi)

In [11]:
def tokenlestir(yorumListesi):
    
    y_yorumlar = []
    for yorum in yorumListesi:
        y_yorum = []
        for kelime in yorum.split():
            if (len(y_yorum) < 50 and kelime in json_tokenizer):
                y_yorum.append(json_tokenizer[kelime])
                
        if (len(y_yorum) < 50):
            sifirlar = list(np.zeros(50 - len(y_yorum), dtype = int))
            y_yorum = sifirlar + y_yorum
            
        y_yorumlar.append(y_yorum)
    return np.array(y_yorumlar, dtype = np.dtype(np.int32))

In [12]:
egitim_kume = tokenlestir(x_train)
test_kume = tokenlestir(x_test)

In [16]:
model = Sequential()

embedding_size = 50

model.add(Embedding(input_dim=10001,
                    output_dim=embedding_size,
                    input_length=50,
                    name='embedding_layer'))

model.add(GRU(units=16, return_sequences=True))
model.add(GRU(units=8, return_sequences=True))
model.add(GRU(units=4))
model.add(Dense(1, activation='sigmoid'))

optimizer = Adam(lr=1e-3)

model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_layer (Embedding)  (None, 50, 50)           500050    
                                                                 
 gru_9 (GRU)                 (None, 50, 16)            3264      
                                                                 
 gru_10 (GRU)                (None, 50, 8)             624       
                                                                 
 gru_11 (GRU)                (None, 4)                 168       
                                                                 
 dense_3 (Dense)             (None, 1)                 5         
                                                                 
Total params: 504,111
Trainable params: 504,111
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
n_y_train = np.array(y_train)
model.fit(egitim_kume, n_y_train, epochs = 15, batch_size = 256)

Epoch 1/15
116/116 [==============================] - 17s 96ms/step - loss: 0.3227 - accuracy: 0.9375
Epoch 2/15
116/116 [==============================] - 11s 95ms/step - loss: 0.2201 - accuracy: 0.9427
Epoch 3/15
116/116 [==============================] - 11s 95ms/step - loss: 0.2181 - accuracy: 0.9427
Epoch 4/15
116/116 [==============================] - 11s 96ms/step - loss: 0.1970 - accuracy: 0.9434
Epoch 5/15
116/116 [==============================] - 11s 97ms/step - loss: 0.1680 - accuracy: 0.9487
Epoch 6/15
116/116 [==============================] - 11s 96ms/step - loss: 0.1404 - accuracy: 0.9558
Epoch 7/15
116/116 [==============================] - 11s 98ms/step - loss: 0.1193 - accuracy: 0.9622
Epoch 8/15
116/116 [==============================] - 11s 98ms/step - loss: 0.1051 - accuracy: 0.9666
Epoch 9/15
116/116 [==============================] - 11s 97ms/step - loss: 0.0941 - accuracy: 0.9709
Epoch 10/15
116/116 [==============================] - 12s 100ms/step - loss: 0.08

In [ ]:
n_y_test = np.array(y_test)
model.evaluate(test_kume, n_y_test)

163/163 [==============================] - 2s 14ms/step - loss: 0.2319 - accuracy: 0.9344


[0.23192021250724792, 0.9344104528427124]

In [ ]:
text1 = "Great tablet for beginners! Very light and screen is wide. Enjoy using for travels.,Love it!"
text2 = "I bought this as a gift for my son and I know he will love it he has the old kindle fire as likes it a lot,Bought it for my son"
text3 = "This Kindle is so overloaded with apps that I will never use. Despite what one of Best Buys' associates said, you cannot remove any of the Amazon apps. The apps take up too much of the memory."
text4 = "Didn't want to go to the sites amazon wanted"
text5 = "Very poor."
text6 = "I am very happy with this Amazon tablet."

yazilar = [text1, text2, text3, text4, text5, text6]

deneme_kume = tokenlestir(yazilar)

sonuc = model.predict(deneme_kume)
sonuc

array([[0.9935616 ],
       [0.99379414],
       [0.01295397],
       [0.9339111 ],
       [0.6719163 ],
       [0.9871743 ]], dtype=float32)

In [ ]:
model.save('/content/gdrive/MyDrive/MachineLearningProject/TrainingModel')

INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/MachineLearningProject/TrainingModel/assets


INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/MachineLearningProject/TrainingModel/assets
